In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
import warnings
warnings.filterwarnings("ignore")
import collections
import random
from tensorflow.keras import layers
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
from matplotlib import pyplot as  plt
from scipy.special import erfinv

In [2]:
#train = np.load('one_hot_seqs.npy')

def id2one_hot(ids):
    res = np.zeros((ids.shape[0], ids.shape[1], 21))
    for i in range(ids.shape[0]):
        for j in range(ids.shape[1]):
            res[i, j, ids[i, j]] = 1
    return res

train = np.array(id2one_hot(np.load('id_seq.npy')))
varient_onehot = np.load('variant_onehot.npy')
varients = pd.read_csv('varient.csv')

#params
logit_sigma = 4.0
logit_p = 0.01
logit_mu = np.sqrt(2.0) * logit_sigma * erfinv(2.0 * logit_p - 1.0)
latent_dim = 30
seq_len = 82
alp_len = 21
sigma_init = -5
def KLD_diag_gaussians(mu, log_sigma, prior_mu, prior_log_sigma):
    return prior_log_sigma - log_sigma + 0.5 * (tf.exp(2. * log_sigma) + tf.square(mu - prior_mu)) * tf.exp(-2.0 * prior_log_sigma) - 0.5

In [3]:
class VarianceLayer(layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units
        
    def Sampler(self, mu, sigma):
        eps = tf.random.normal(sigma.shape)
        return mu + eps * (tf.exp(sigma)**0.5)    
        
    def build(self, input_shape):
        self.wmu = self.add_variable(name='wmu',shape=[input_shape[-1], self.units],
                                    initializer=tf.random_normal_initializer(0, tf.sqrt(2.0/(input_shape[-1] + self.units))))
        self.wsigma = self.add_variable(name='wsigma',shape=[input_shape[-1], self.units], initializer=tf.constant_initializer(sigma_init))
        self.bmu = self.add_variable(name='bmu',shape=[self.units], initializer=tf.constant_initializer(0.1))
        self.bsigma = self.add_variable(name='bsigma',shape=[self.units], initializer=tf.constant_initializer(sigma_init))
    
    def KLD_params(self):
        res = - tf.reduce_mean(KLD_diag_gaussians(self.wmu, self.wsigma, 0, 0.))
        res -= tf.reduce_mean(KLD_diag_gaussians(self.bmu, self.bsigma, 0, 0.)) 
        return res
    
    
    def call(self, inputs, var=True):     
        w = self.Sampler(self.wmu, self.wsigma)
        b = self.Sampler(self.bmu, self.bsigma)
        output = tf.matmul(inputs, w) + b
        
        return output

In [4]:
class OutputLayer(layers.Layer):
    def __init__(self, units, seq_len, n_pat = 4):
        super().__init__()
        self.units = units
        self.seq_len = seq_len
        self.n_pat = n_pat
        
        
    def Sampler(self, mu, sigma):
        eps = tf.random.normal(sigma.shape)
        return mu + eps * (tf.exp(sigma)**0.5)    
        
    def build(self, input_shape):
        self.inputshape = input_shape
        self.w_out_mu = self.add_variable(name='w_out_mu',shape=[input_shape[-1] * self.seq_len, self.units], 
                                          initializer=tf.random_normal_initializer(0, tf.sqrt(2.0/(input_shape[-1] * self.seq_len+self.units))))
        self.w_out_sigma = self.add_variable(name='w_out_sigma',shape=[input_shape[-1] * self.seq_len, self.units], 
                                             initializer=tf.constant_initializer(sigma_init))
        
        self.w_conv_mu = self.add_variable(name='w_conv_mu',shape=[self.units, alp_len], 
                                           initializer=tf.random_normal_initializer(0, tf.sqrt(2.0/(self.units*alp_len))))
        self.w_conv_sigma = self.add_variable(name='w_conv_sigma',shape=[self.units, alp_len], 
                                              initializer=tf.constant_initializer(sigma_init))
        
        self.w_scale_mu = self.add_variable(name='w_scale_mu',shape=[int(input_shape[-1] / self.n_pat), self.seq_len], 
                                            initializer=tf.random_normal_initializer(0, tf.sqrt(2.0/(int(input_shape[-1] / self.n_pat)*self.seq_len))))
        self.w_scale_sigma = self.add_variable(name='w_scale_sigma',shape=[int(input_shape[-1] / self.n_pat), self.seq_len], 
                                               initializer=tf.constant_initializer(sigma_init))
        
        self.bmu = self.add_variable(name='bmu',shape=[self.seq_len * alp_len], 
                                     initializer=tf.random_normal_initializer(0.1))
        self.bsigma = self.add_variable(name='bsigma',shape=[self.seq_len * alp_len], 
                                        initializer=tf.constant_initializer(sigma_init))
        self.psw_mu = self.add_variable(name='psw_mu', shape=[1], initializer=tf.constant_initializer(1))
        self.psw_sigma = self.add_variable(name='psw_sigma', shape=[1], initializer=tf.constant_initializer(sigma_init))
    
    
    def FadeOut(self):
        return tf.reduce_mean(KLD_diag_gaussians(self.w_scale_mu, self.w_scale_sigma, logit_mu, tf.math.log(logit_sigma)))
    
    
    def KLD_params(self):
        res = - tf.reduce_mean(KLD_diag_gaussians(self.w_conv_mu, self.w_conv_sigma, 0, 0))
        res -= tf.reduce_mean(KLD_diag_gaussians(self.w_out_mu, self.w_out_sigma, 0, 0))
        res -= tf.reduce_mean(KLD_diag_gaussians(self.bmu, self.bsigma, 0, 0))
        return res - self.FadeOut()
     
    
    
    def call(self, inputs):
        w_out = self.Sampler(self.w_out_mu, self.w_out_sigma)
        w_conv = self.Sampler(self.w_conv_mu, self.w_conv_sigma)
        w_out = tf.matmul(w_out, w_conv)
        w_scale = self.Sampler(self.w_scale_mu, self.w_scale_sigma)
        w_scale = tf.tile(w_scale, (self.n_pat, 1))
        w_scale = tf.expand_dims(w_scale, -1)
        w_out = tf.reshape(w_out, (self.inputshape[-1], self.seq_len, alp_len))
        w_out = tf.multiply(w_out, w_scale)
        w_out = tf.reshape(w_out, (self.inputshape[-1], self.seq_len * alp_len))
        
        b = self.Sampler(self.bmu, self.bsigma)
        output = tf.matmul(inputs, w_out) + b
        psw = self.Sampler(self.psw_mu, self.psw_sigma)
        return tf.multiply(output, tf.math.log(1.0 + tf.exp(psw)))

In [5]:
class Deep_Seq(tf.keras.Model):
    def __init__(self):
        super(Deep_Seq, self).__init__()

        
        self.flat1 = layers.Flatten()
        self.encoder1 = layers.Dense(1500)
        
        self.encoder2 = layers.Dense(1500)
        
        self.hmu = layers.Dense(latent_dim)
        self.hsigma = layers.Dense(latent_dim)
        
        self.decoder1 = VarianceLayer(100)
        self.decoder2 = VarianceLayer(500)

        self.out = OutputLayer(10, seq_len)
        
        self.reshape = layers.Reshape((seq_len, alp_len))
        
        
    def Encoder(self, x):
        x = self.flat1(x)
        x = self.encoder1(x)
        x = tf.nn.relu(x)
        x = self.encoder2(x)
        x = tf.nn.relu(x)
        mu = self.hmu(x)
        sigma = self.hsigma(x)
        
        return mu, sigma
    
    def Sampler(self, mu, sigma):
        eps = tf.random.normal(sigma.shape)
        return mu + eps * (tf.exp(sigma)**0.5)
    
    def Decoder(self, z):
        z = self.decoder1(z)
        z = tf.nn.relu(z)
        
        z = self.decoder2(z)
        z = tf.nn.relu(z)

        x_hat = self.out(z)
        x_hat = self.reshape(x_hat)
        
        return x_hat
    
    def KLD_params(self):
        return self.decoder1.KLD_params() + self.decoder2.KLD_params()+ self.out.KLD_params()
        #
    
    def call(self, x):
        mu, sigma = self.Encoder(x)
        z = self.Sampler(mu, sigma)
        x_hat = self.Decoder(z)
        
        return x_hat, mu, sigma


In [6]:
vae = Deep_Seq()#model
batch_size = 512
optimizer = tf.optimizers.Adam()
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.repeat().shuffle(1000).batch(batch_size).prefetch(1)


##########################################################################################


train_step = 100000
display_step = 500
losses = []
corrs = []
for step, batch_x in enumerate(train_data.take(train_step + 1)):
   
    with tf.GradientTape() as tape:
        x_, mu, sigma = vae(batch_x)
        kl_div = -0.5 * (2 * sigma + 1 - tf.square(mu) - tf.exp(2 * sigma))
        kl_div = tf.reduce_mean(kl_div)
        ce = tf.nn.softmax_cross_entropy_with_logits(batch_x, x_)
        loss = tf.reduce_mean(ce) + kl_div - vae.KLD_params()
    gradient = tape.gradient(loss, vae.trainable_variables)
    optimizer.apply_gradients(zip(gradient, vae.trainable_variables))
    
    if step % display_step == 0:
        
        pre, mu, sigma= vae(varient_onehot)
        kl_div = tf.reduce_mean(-0.5 * (2 * sigma + 1 - tf.square(mu) - tf.exp(2 * sigma)), axis=1).numpy()
        elbo = - tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(varient_onehot, pre), axis=1).numpy() - kl_div
        pre_wt, mu, sigma= vae(train[0:1])
        kl_div = tf.reduce_mean(-0.5 * (2 * sigma + 1 - tf.square(mu) - tf.exp(2 * sigma)), axis=1).numpy()
        wt_elbo = - tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(train[0:1], pre_wt)) - kl_div
        
        varients['pre'] = elbo - wt_elbo.numpy()[0]
        corr = varients.corr('spearman').values[1][-1]
        
        corrs.append(corr)
        losses.append(loss.numpy())
        print("step: %i, loss: %f, spearman_corr: %f "  % (step, loss, corr))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Please use `layer.add_weight` method instead.
step: 0, loss: 43.224094, spearman_corr: -0.030865 
step: 500, loss: 38.361443, spearman_corr: 0.276990 
step: 1000, loss: 34.722839, spearman_corr: 0.183478 
step: 1500, loss: 31.053459, spearman_corr: 0.266248 
step: 2000, loss: 27.416258, spearman_corr: 0.217077 
step: 2500, loss: 23.931871, spearman_corr: 0.154710 
step: 3000, loss: 20.442411, spearman_corr: 0.075791 
step: 3500, loss: 17.129961, spearman_corr: 0.172258 
step: 

KeyboardInterrupt: 